In [1]:
import scipy as sp
import pylab as plt
from scipy.integrate import odeint
import numpy as np
import random

In [2]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}

plt.rc('font', **font)

In [6]:
# Constants
C_m = 1.0
g_Na = 120
g_K = 36.0
g_L = 0.3
E_Na = 45.0
E_K = -82.0
E_L = -59.387
E_Cl = -80.0
E_ex = -38.0

# channel gating kinetics
def a_m(V): return (25-V)/(10*(np.exp((25-V)/10)-1))
def b_m(V): return 4*np.exp(-V/18)
def a_h(V): return 0.07*np.exp(-V/20)
def b_h(V): return 1/(np.exp((30-V)/10)+1)
def a_n(V): return (10-V)/(100*(np.exp((10-V)/10)-1))
def b_n(V): return 0.125*np.exp(-V/80)

In [7]:
# alpha/beta constants for ex/inhib synapses
a_r_i = 5.0
b_r_i = 0.18
a_r_e = 2.4
b_r_e = 0.56

# [T] equations for synapses
T_max_i = 1.5
T_max_e = 1.0
K_p = 5.0
V_p = 7.0

def T_i(V_pre): return T_max_i/(1+np.exp(-(V_pre-V_p)/K_p)) 
def T_e(V_pre): return T_max_e/(1+np.exp(-(V_pre-V_p)/K_p)) 

In [8]:
# Differential gating equations
def dm(V,m): return a_m(V)*(1.0-m)-b_m(V)*m
def dh(V,h): return a_h(V)*(1.0-h)-b_h(V)*h
def dn(V,n): return a_n(V)*(1.0-n)-b_n(V)*n
def dr_i(V,r): return a_r_i*T_i(V)*(1.0-r)-b_r_i*r
def dr_e(V,r): return a_r_e*T_e(V)*(1.0-r)-b_r_e*r

In [9]:
# Membrane currents (uA/cm^2)
def I_Na(V,m,h): return g_Na*m**3*h*(V-E_Na)
def I_K(V,n): return g_K*n**4*(V-E_K)
def I_L(V): return g_L*(V-E_L)

def I_syn_i(V,g_GABA,r): return r*g_GABA*(V-E_Cl)
def I_syn_e(V,g_Glu,r): return r*g_Glu*(V-E_ex)

In [11]:
# membrane voltage differential 
def dV(V,m,h,n,r_i,r_e,I_ext,g_GABA,g_Glu): return (-I_Na(V,m,h) - I_K(V,n)-I_L(V)-I_syn_i(V,g_GABA,r_i)-I_syn_e(V,g_Glu,r_e)+I_ext)/C_m
# X- column -> cells, rows -> parameters
# g_GABA (inhibitory synapses) rows -> presynaptic cell, column -> postsynaptic cell
def d_single(X,t,I_ext,g_GABA,g_Glu):
    V,m,h,n,r_i,r_e = X[0,:],X[1,:],X[2,:],X[3,:],X[4,:],X[5,:]
    return sp.array([
            dV(V,m,h,n,r_i,r_e,I_ext(t),g_GABA,g_Glu),
            dm(V,m), dh(V,h), dn(V,n),
            dr_i(V,r_i), dr_e(V,r_e)
        ])

In [12]:
# multiple Neurons
# I_exts takes time and returns row vector of current applied to each cell
def d(X,t,I_exts,g_GABA,g_Glu):
    return sp.reshape(d_single(sp.reshape(X,(6,len(X)/6)),t,I_exts,g_GABA,g_Glu),len(X))

# simulate whole network
t = sp.linspace(0,500,5000) # 0.1 ms / div
def network(I_exts,g_GABA,g_Glu):
    n_cells = sp.size(g_GABA,0)
    return sp.reshape(
    odeint(d,sp.repeat([-70,.05,0.6,0.3,0,0],ncells),t,(I_exts,g_GABA,g_Glu)).T,(6,ncells,len(t)))
# no connection network
nocon2 = sp.zeros((2,2))

In [19]:
# PART 1
I_exts_ = [10,20]
def I_exts(t): return I_exts_


In [18]:
foo = [10,20]
bar = (10,20)
x =5 
type(x)

int